In [313]:
import numpy as np
import torch
import torch.nn.functional as F

In [314]:
data = open('names.txt').read().splitlines()
data[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [315]:
chars = sorted(list({ch for w in data for ch in w}))

# First try for trigram

In [145]:
# Used 27*27=729 input dimension for input of two characters
# doesnt work well though ... try to think why this isnt good. Drops information?

In [316]:
chars = sorted(list({ch for w in data for ch in w}))
chars = ['.'] + chars
ytoi = {s:i for i,s in enumerate(chars)}
ytos = {i:s for s,i in ytoi.items()}
charcount=len(stoi)

In [317]:
xtoi = {}
i = 0
for ch1 in chars:
    for ch2 in chars: 
        xtoi[(ch1, ch2)] = i
        i += 1
        
xtos = {i:s for s,i in xtoi.items()}
xdim = len(xtoi)

In [320]:
# build a trigram model

# create the training set of bigrams
xs, ys = [], []
for w in data:
    chs = ['.'] + ['.'] + list(w) + ['.'] + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix = xtoi[(ch1, ch2)]
        iy = ytoi[ch3]
        xs.append(ix)
        ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

# randomly initialise 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((xdim, 27), generator=g, requires_grad=True)

number of examples:  260179


In [321]:
# print([itos[x[0].item()] + itos[x[1].item()] for x in xs[:6]])
print([xtos[x.item()] for x in xs[:6]])
print([ytos[y.item()] for y in ys[:6]])

[('.', '.'), ('.', 'e'), ('e', 'm'), ('m', 'm'), ('m', 'a'), ('a', '.')]
['e', 'm', 'm', 'a', '.', '.']


In [322]:
# gradient descend
for k in range(100):
    # forward pass
    xenc = F.one_hot(xs, num_classes=xdim).float() # input to the network: one_hot encoding
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilites for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set the gradients to zero
    loss.backward()
    
    # update
    W.data += -50 * W.grad

3.7422749996185303
3.5306496620178223
3.3819174766540527
3.2758371829986572
3.1978492736816406
3.1354622840881348
3.082799196243286
3.0368664264678955
2.995882987976074
2.958749294281006
2.924757480621338
2.8934175968170166
2.8643672466278076
2.8373255729675293
2.8120646476745605
2.788397789001465
2.7661678791046143
2.745241641998291
2.725506067276001
2.706862449645996
2.689223289489746
2.6725120544433594
2.656658887863159
2.6416001319885254
2.627277374267578
2.613636016845703
2.6006274223327637
2.5882060527801514
2.576331377029419
2.5649654865264893
2.554074287414551
2.543628215789795
2.533597946166992
2.5239593982696533
2.514688491821289
2.5057642459869385
2.497166872024536
2.488878011703491
2.4808812141418457
2.4731600284576416
2.465700149536133
2.4584872722625732
2.4515087604522705
2.4447524547576904
2.438206672668457
2.4318606853485107
2.4257047176361084
2.419728994369507
2.4139249324798584
2.4082834720611572
2.4027976989746094
2.3974592685699463
2.3922622203826904
2.3871996402740

In [144]:
# Sample from the neutral net

g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out=[]
    ix = 0
    while True:
        # p = P[ix]
        xenc = F.one_hot(torch.tensor([ix]), num_classes=xdim).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(ytos[ix])
        if ix == 0: 
            break
    print(''.join(out))

emracvvaromkxttszpgschuhqvbmxemc.
emmxqvydusmasemripdkfbllvkfemhusxnumhtdjixwlu.
embw.
emhpyucemhqhrxktpdiqslu.
emajadkrlqriabwtvscjmhqwtpxemygdwfvnnjcjchwzwjywtogdnppollwtqyhdmwdjuyggluyyyqwvbuytofbcvbmbotdkajlnplpnpjmjmaoodqy.


# Second try for trigram

In [ ]:
# Not sure if this is a trigram model. Not taking order into account here. 
# a b -> c    and   b a -> c      is the same

In [333]:
chars = sorted(list({ch for w in data for ch in w}))
chars = ['.'] + chars
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
numchars=len(chars)

In [335]:
# stoi

In [323]:
# build a trigram model

# create the training set of bigrams
xs, ys = [], []
for w in data:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        iy = ytoi[ch3]
        xs.append((ix1, ix2))
        ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

# randomly initialise 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2, 27), generator=g, requires_grad=True)

number of examples:  196113


In [324]:
print(xs[:6])
print(ys[:6])
print([itos[x[0].item()] + itos[x[1].item()] for x in xs[:6]])
print([itos[y.item()] for y in ys[:6]])

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1],
        [ 0, 15],
        [15, 12]])
tensor([13, 13,  1,  0, 12,  9])
['.e', 'em', 'mm', 'ma', '.o', 'ol']
['m', 'm', 'a', '.', 'l', 'i']


In [377]:
# Check reshaping ... 
#xenc = F.one_hot(torch.tensor([0, 0]), num_classes=3).float()
#xenc.view(-1, 2*3)

In [378]:
# gradient descend
for k in range(100):
    # forward pass
    xenc = F.one_hot(xs, num_classes=numchars).float() # input to the network: one_hot encoding
    logits = xenc.view(-1, 2*27) @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilites for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set the gradients to zero
    loss.backward()
    
    # update
    W.data += -1 * W.grad

2.243431329727173
2.2434310913085938
2.2434303760528564
2.2434298992156982
2.24342942237854
2.2434284687042236
2.2434279918670654
2.2434275150299072
2.243427038192749
2.2434263229370117
2.2434256076812744
2.243425130844116
2.243424654006958
2.2434241771698
2.2434234619140625
2.2434229850769043
2.243422269821167
2.243421792984009
2.2434213161468506
2.2434206008911133
2.243420124053955
2.2434194087982178
2.2434189319610596
2.2434182167053223
2.243417739868164
2.2434170246124268
2.2434163093566895
2.2434158325195312
2.243415355682373
2.243414878845215
2.2434141635894775
2.2434139251708984
2.243412971496582
2.243412494659424
2.2434120178222656
2.2434113025665283
2.24341082572937
2.243410348892212
2.2434096336364746
2.2434091567993164
2.243408441543579
2.243408203125
2.2434074878692627
2.2434067726135254
2.243406295776367
2.243405818939209
2.2434051036834717
2.2434043884277344
2.243403911590576
2.243403434753418
2.2434027194976807
2.2434022426605225
2.243401527404785
2.243401050567627
2.243

In [379]:
# Sample from the neutral net

g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out=[]
    ix1 = 0
    ix2 = torch.randint(1, 27, (1,), generator=g)
    while True:
        # p = P[ix]
        xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=numchars).float()
        logits = xenc.view(-1, 2*27) @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)
        
        iy = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(ytos[iy])
        if iy == 0: 
            break
            
        ix1 = ix2
        ix2 = iy
    print(''.join(out))

osprgetrumallymarlannavikh.
ree.
isnerse.
ore.
sarneve.
ah.
aim.
enikavielah.
sh.
aia.
